In [1]:
import openai
import json

client = openai.OpenAI()
messages = []

In [2]:
def get_weather(city):
    return "33 degrees celcius."


FUNCTION_MAP = {
    "get_weather": get_weather,
}

In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of. Whole words must be translate to english",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [4]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):

    if message.tool_calls and len(message.tool_calls) > 0:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling functionL: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)
    message = response.choices[0].message.tool_calls

In [5]:
while True:
    message = input("LLM에게 메세지를 보내세요...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 안녕 난 rearleg야.
AI: 안녕하세요, rearleg님! 어떻게 도와드릴까요?
User: 오늘 서울 날씨를 보고 낮 활동에 대한 팁을 줘.
Calling functionL: get_weather with {"city":"Seoul"}
Ran get_weather with args {'city': 'Seoul'} for a result of 33 degrees celcius.
AI: 오늘 서울의 날씨는 33도 섭씨입니다. 이렇게 더운 날씨에는 다음과 같은 낮 활동 팁을 드립니다:

1. **수분 섭취**: 더운 날씨에는 탈수될 위험이 있으니 충분한 물을 마시는 것이 중요합니다.
2. **햇볕 피하기**: 가능한 한 그늘에서 활동하거나, 햇볕이 강한 시간대(정오~오후 3시)는 피하는 것이 좋아요.
3. **가벼운 옷차림**: 통기성이 좋은 가벼운 옷을 입어 더위를 이길 수 있도록 하세요.
4. **적절한 활동 계획**: 실내에서 할 수 있는 활동이나 수영과 같은 시원한 활동을 고려해보세요.
5. **선크림 바르기**: 피부를 보호하기 위해 외출 전 선크림을 꼭 바르는 것이 좋습니다.

안전하고 즐거운 활동 되세요!
User: 죽음을 각오했나?
AI: 죽음을 각오했다는 표현은 조금 극단적이네요. 어떤 걱정이나 고민이 있으신가요? 도움이 필요하시면 말씀해 주세요.
User: 크하하하 난 흑염룡이다. 내 파이어볼을 맞아볼테냐?
AI: 크하하하! 흑염룡님, 당신의 파이어볼은 무시무시하군요. 하지만 저는 무적이니까요! 어떤 모험을 함께 하실 건가요?
User: 오늘 날씨 알려줘
Calling functionL: get_weather with {"city":"Seoul"}
Ran get_weather with args {'city': 'Seoul'} for a result of 33 degrees celcius.
AI: 오늘 서울의 날씨는 33도 섭씨입니다. 무척 덥네요! 흑염룡님, 이런 날씨에 어떤 계획이 있으신가요?
User: 내 이름이 뭐게?
AI: 

In [6]:
messages

[{'role': 'user', 'content': '안녕 난 rearleg야.'},
 {'role': 'assistant', 'content': '안녕하세요, rearleg님! 어떻게 도와드릴까요?'},
 {'role': 'user', 'content': '오늘 서울 날씨를 보고 낮 활동에 대한 팁을 줘.'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_pPAYEmUsdPtfaPtM63VdsnoA',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Seoul"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_pPAYEmUsdPtfaPtM63VdsnoA',
  'name': 'get_weather',
  'content': '33 degrees celcius.'},
 {'role': 'assistant',
  'content': '오늘 서울의 날씨는 33도 섭씨입니다. 이렇게 더운 날씨에는 다음과 같은 낮 활동 팁을 드립니다:\n\n1. **수분 섭취**: 더운 날씨에는 탈수될 위험이 있으니 충분한 물을 마시는 것이 중요합니다.\n2. **햇볕 피하기**: 가능한 한 그늘에서 활동하거나, 햇볕이 강한 시간대(정오~오후 3시)는 피하는 것이 좋아요.\n3. **가벼운 옷차림**: 통기성이 좋은 가벼운 옷을 입어 더위를 이길 수 있도록 하세요.\n4. **적절한 활동 계획**: 실내에서 할 수 있는 활동이나 수영과 같은 시원한 활동을 고려해보세요.\n5. **선크림 바르기**: 피부를 보호하기 위해 외출 전 선크림을 꼭 바르는 것이 좋습니다.\n\n안전하고 즐거운 활동 되세요!'},
 {'role': 'user', 'content': '죽음을 각오했나?'},
 {'role': 'assistant',
  'content': '죽음을 각오했다